## Notebook 2: Data Preparation

This notebook focuses on preprocessing the raw data based on insights from the initial EDA (`01_initial_eda.ipynb`).

In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer


# Setup
np.random.seed(42)
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

print("Setup complete.")

Setup complete.


### 1. Load Raw Data

In [2]:
data_path = 'Clustering_Data.ftr'


df = pd.read_feather(data_path)
print(f"Dataset loaded successfully: {data_path}")


print(f"Initial shape: {df.shape}")

Dataset loaded successfully: Clustering_Data.ftr
Initial shape: (3030, 18)


In [3]:
df.head()

,Customer_ID,Street,City,State_Code,Postal_Code,Country,Seasonality_Segment,EA_Segment,Revenue_Bucket,Profit_Bucket,Market_Share_Segment,Casino_Size_Segment,Market_Potential_Segment,Churn_Segment,Competitiveness_Flag,Volume_Segment,Density_Segment,Propensity
0,Accoount 1,2295 BATTLEFORD RD,MISSISSAUGA,ON,L5N 2W8,CAN,None,None,None,None,None,None,None,None,Yes,High,High,-
1,Accoount 2,64 WESTBANK EXPY STE A,GRETNA,LA,70053,USA,None,None,None,None,None,None,None,None,-,Medium,Medium,-
2,Accoount 3,209-229 E Airline Hwy.,LaPlace,LA,70068,USA,None,None,None,None,None,None,None,None,-,Medium,Medium,-
3,Accoount 4,500 S MEADOWS PKWY,RENO,NV,89521,USA,None,None,None,None,None,None,None,None,Yes,High,High,-
4,Accoount 5,746 Victorian Avenue,Sparks,NV,89431,USA,None,None,None,None,None,None,None,None,Yes,High,High,-


### 2. Drop Irrelevant Columns

In [4]:
columns_to_drop = ['Street', 'City', 'State_Code', 'Postal_Code', 'Country']
df.drop(columns=columns_to_drop, inplace=True)

print("Shape after dropping irrelevant columns:", df.shape)


Shape after dropping irrelevant columns: (3030, 13)


### 3. Clean and Standardize Categorical Values


In [5]:
# Columns to clean and encode
categorical_columns = [
    'Seasonality_Segment', 'EA_Segment', 'Revenue_Bucket', 'Profit_Bucket',
    'Market_Share_Segment', 'Casino_Size_Segment','Market_Potential_Segment',
    'Churn_Segment', 'Competitiveness_Flag', 'Volume_Segment', 'Density_Segment', 'Propensity'
]

# Replace None, 'None', '-', NaN with 'Missing'
for col in categorical_columns:
    print(f"\nOriginal unique values in '{col}': {df[col].unique()}")
    df[col] = df[col].replace([None, 'None', '-', np.nan], 'Missing')
    print(f"Standardized unique values in '{col}': {df[col].unique()}")



Original unique values in 'Seasonality_Segment': ['None' None 'Highly Seasonal' 'Potentially Seasonal' 'Non Seasonal']
Standardized unique values in 'Seasonality_Segment': ['Missing' 'Highly Seasonal' 'Potentially Seasonal' 'Non Seasonal']

Original unique values in 'EA_Segment': ['None' None 'Early Adopter' 'Late Adopter' 'Trend Follower']
Standardized unique values in 'EA_Segment': ['Missing' 'Early Adopter' 'Late Adopter' 'Trend Follower']

Original unique values in 'Revenue_Bucket': ['None' 'L' 'M' 'H']
Standardized unique values in 'Revenue_Bucket': ['Missing' 'L' 'M' 'H']

Original unique values in 'Profit_Bucket': ['None' 'H' 'L' 'M']
Standardized unique values in 'Profit_Bucket': ['Missing' 'H' 'L' 'M']

Original unique values in 'Market_Share_Segment': ['None' 'L' 'M' 'H']
Standardized unique values in 'Market_Share_Segment': ['Missing' 'L' 'M' 'H']

Original unique values in 'Casino_Size_Segment': ['None' 'L' 'M' 'H']
Standardized unique values in 'Casino_Size_Segment': ['Mi

### 4. Merge Columns acc to results of Cramer's V

In [6]:
# Find customer IDs where Casino_Size_Segment is Missing
missing_casino_ids = set(df.loc[df['Casino_Size_Segment'] == 'Missing', 'Customer_ID'])

# Find customer IDs where Market_Potential_Segment is Missing
missing_potential_ids = set(df.loc[df['Market_Potential_Segment'] == 'Missing', 'Customer_ID'])

# Compare sets
if missing_casino_ids == missing_potential_ids:
    print("✅ Missing values occur for the exact same Customer_IDs in both columns.")
else:
    print("⚠️ Different Customer_IDs have missing values in these two columns.")
    print(f"Customers missing in Casino_Size_Segment only: {len(missing_casino_ids - missing_potential_ids)}")
    print(f"Customers missing in Market_Potential_Segment only: {len(missing_potential_ids - missing_casino_ids)}")


✅ Missing values occur for the exact same Customer_IDs in both columns.


In [7]:
# Categories to consider:
# Low → both L
# Medium → one M, other M/L
# High → one H, other H/VH
# Exceptional → any instance of VH
# Missing → if either is missing

def classify_casino_potential(size, potential):
    if size == 'Missing' or potential == 'Missing':
        return 'Missing'
    elif 'VH' in [potential]:
        return 'Exceptional'
    elif size == 'H' or potential in ['H']:
        return 'High'
    elif size == 'M' or potential == 'M':
        return 'Medium'
    elif size == 'L' and potential == 'L':
        return 'Low'

df['Casino_Potential_Class'] = df.apply(
    lambda row: classify_casino_potential(row['Casino_Size_Segment'], row['Market_Potential_Segment']),
    axis=1
)


In [16]:
df.drop(columns=['Market_Potential_Segment', 'Casino_Size_Segment'], inplace=True)


In [17]:
df.sample(5)

,Customer_ID,Seasonality_Segment,EA_Segment,Revenue_Bucket,Profit_Bucket,Market_Share_Segment,Churn_Segment,Competitiveness_Flag,Volume_Segment,Density_Segment,Propensity,Casino_Potential_Class
131,Accoount 132,1,2,1.0,3.0,2.0,2,0,2.0,3.0,2.0,1.0
869,Accoount 870,1,2,0.0,0.0,0.0,3,1,3.0,3.0,0.0,0.0
2133,Accoount 2134,1,2,0.0,0.0,0.0,3,1,3.0,1.0,0.0,0.0
2012,Accoount 2013,1,2,0.0,0.0,0.0,3,0,1.0,3.0,0.0,0.0
2845,Accoount 2846,1,2,0.0,0.0,0.0,3,0,0.0,0.0,0.0,0.0


In [11]:
df['Casino_Potential_Class'].unique()

array(['Missing', 'Medium', 'High', 'Low', 'Exceptional'], dtype=object)

### 5. Ordinal Encoding Columns

In [12]:
ordinal_mappings = {
    'Revenue_Bucket': ['Missing', 'L', 'M', 'H'],
    'Profit_Bucket': ['Missing', 'L', 'M', 'H'],
    'Propensity': ['Missing', 'L', 'M', 'H'],
    'Market_Share_Segment': ['Missing', 'L', 'M', 'H'],
    'Volume_Segment': ['Missing', 'Low', 'Medium', 'High'],
    'Casino_Potential_Class': ['Missing', 'Medium', 'High', 'Low', 'Exceptional'],
    'Density_Segment': ['Missing', 'Low', 'Medium', 'High']
}

# Apply OrdinalEncoder with explicit category order
ordinal_cols = list(ordinal_mappings.keys())
encoder = OrdinalEncoder(categories=[ordinal_mappings[col] for col in ordinal_cols])
df[ordinal_cols] = encoder.fit_transform(df[ordinal_cols])

### 6. Label Encode Categorical Columns

In [13]:
label_cols = [
    'Seasonality_Segment', 'EA_Segment', 'Churn_Segment', 'Competitiveness_Flag'
]

le = LabelEncoder()
for col in label_cols:
    df[col] = le.fit_transform(df[col])

### 7. Prepare Final Data for Clustering

In [14]:
print("Final cleaned data shape:", df.shape)
df.head()

Final cleaned data shape: (3030, 14)


,Customer_ID,Seasonality_Segment,EA_Segment,Revenue_Bucket,Profit_Bucket,Market_Share_Segment,Casino_Size_Segment,Market_Potential_Segment,Churn_Segment,Competitiveness_Flag,Volume_Segment,Density_Segment,Propensity,Casino_Potential_Class
0,Accoount 1,1,2,0.0,0.0,0.0,Missing,Missing,3,1,3.0,3.0,0.0,0.0
1,Accoount 2,1,2,0.0,0.0,0.0,Missing,Missing,3,0,2.0,2.0,0.0,0.0
2,Accoount 3,1,2,0.0,0.0,0.0,Missing,Missing,3,0,2.0,2.0,0.0,0.0
3,Accoount 4,1,2,0.0,0.0,0.0,Missing,Missing,3,1,3.0,3.0,0.0,0.0
4,Accoount 5,1,2,0.0,0.0,0.0,Missing,Missing,3,1,3.0,3.0,0.0,0.0


In [ ]:
df.reset_index(drop=True).to_feather("outputs/preprocessed_data_final.ftr")